In [4]:
"""
This is the most basic script for using twitter_dm.

From here, you may want to go look at some of the more complex examples
that leverage the library's NLP/rapid collection tools, as this is basically
a replication of tweepy with less documentation :)
"""
from twitter_dm.TwitterAPIHook import TwitterAPIHook
from twitter_dm.TwitterUser import TwitterUser

username_to_collect_data_for = 'buffalobills'

consumer_key = "YOUR_KEY"
consumer_secret = "YOUR_SECRET"
access_token = "YOUR_ACCESS_TOKEN"
access_token_secret = "YOUR_ACCESS_SECRET"

## get a "hook", or connection, to the API using your consumer key/secret and access token/secret
api_hook = TwitterAPIHook(consumer_key,consumer_secret,
                          access_token=access_token,access_token_secret=access_token_secret)

#creates a Twitter User object to fill with information from the API
user = TwitterUser(api_hook,screen_name=username_to_collect_data_for)


# we call populate_tweets_from_api,which goes to the Twitter API
# and collects the user's data it is outputted to the file username_you_put.json
# the sleep_var param tells the function it shouldn't worry
# about rate limits (we're only collecting for one user, so it doesn't really matter
# If you remove the is_gzip argument, the output file will be gzipped
print 'populating users tweets!'
user.populate_tweets_from_api(json_output_filename=username_to_collect_data_for+".json",
                              sleep_var=False, is_gzip=False, since_id=None)



populating users tweets!
539  total tweets for:  _kenny_joseph   539  new tweets from API
[189837839]
[14454727]
[14454727]
[]
[339348373]
[339348373]
[]
[370723081]
[]
[14454727]
[]
[14454727]
[]
[14454727, 204586625]
[]
[175025887]
[175025887]
[53823591]
[244432909]
[]
[]
[]
[76055533, 65804986]
[244432909, 14454727]
[1290351, 9866582]
[]
[]
[]
[]
[370723081]
[]
[]
[116980216]
[]
[]
[368634919]
[2955801095L]
[175025887]
[156170258]
[53823591, 778744200]
[116980216]
[]
[15207385]
[1200985080]
[368634919]
[1200985080]
[]
[]
[523895367, 370723081]
[]
[370723081, 368634919]
[]
[13447]
[]
[25084916]
[14773649]
[1012060890169004032L]
[661613, 15166546]
[368634919]
[53823591, 371066122]
[18463930]
[1200985080]
[]
[]
[390550654]
[175025887]
[1012060890169004032L, 1200985080]
[235700566, 405033226]
[105530526, 298729878]
[579299426, 14299159]
[]
[]
[370723081]
[]
[78779375]
[31615214, 15166546, 33786195, 56843433, 20963651]
[117171240]
[415601652]
[]
[143449579]
[415601652]
[18463930]
[117641

In [7]:
for t in user.tweets:
    print t.tokens
print 'user had {n_tweets} tweets'.format(n_tweets=len(user.tweets))


[u'usevolleyball', u'guy', u'thanks', u'help']
[u'bloomt', u'beautiful', u'ham', u'sandwich', u'bambiphi']
[u'bloomt', u'decided', u'fat', u'fast', u'eating', u'food', u'fasting', u'watching', u'football', u'checking', u'fantasy', u'whichisworse']
[u'sweet', u'old', u'lady', u'holding', u'door', u'robot', u'cmu', u'pretty', u'confused']
[u'tjgiles2', u'hate', u'player', u'hate', u'game']
[u'tjgiles2']
[u'mugging', u'weekend', u'block', u'apartment', u'terrifying', u'attack', u'dog', u'scarychihuahuainasweater']
[u'mikeberkun', u'heywood', u'swing', u'crush', u'towrd', u'south', u'america', u'tomlinson', u'gonna', u'visa', u'catch']
[u'jpmon_energy', u'pa', u'dep', u'authorize', u'mine', u'water', u'fracking', u'policy', u'address', u'historic', u'mine', u'drainage', u'reduce', u'water']
[u'bloomt', u'team', u'chara', u'team', u'alfredsson', u'all-star', u'game', u'sound', u'european', u'pokemon', u'battle', u'marketingfail']
[u'cooking', u'fun', u'pretend', u'chopped', u'secret', u'ing

In [5]:
import tweepy

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print tweet.text

RT @ESPNNFL: The @Patriots donated 200 beds for 200 kids in need. https://t.co/6bz0uyE9n0
BIZARRE: Police say a man flew a small plane into his own house just hours after being arrested for assaulting his… https://t.co/lAuFrmQ66Y
Coach Tomlin addresses the media following practice at #SteelersCamp. https://t.co/rQ6OHyi8bI
Ray Fittipaldo's Steelers chat transcript: 8.13.18 https://t.co/DxYXPEmHCL
Can't. Look. Away. https://t.co/L8Nbppgehb
A group of Chinese hackers spent months developing a new technique for hijacking Amazon's voice assistant gadget. B… https://t.co/fQZ5CLFeMg
In theaters soon. https://t.co/V9kDoq1PFB
West Virginia's House of Delegates has voted to impeach three of the four justices on the state's Supreme Court of… https://t.co/DUFfpq4dOy
“Biohacking isn’t just about physical gains. It’s also about mental clarity, feeling energized, and even making pea… https://t.co/haZ1e5TIv5
There’s an underwater pokéball that helps us study delicate sea creatures without harming them